In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
def load_latest_indicator(path, indicator_name, value_col_name):
    """
    Wczytuje najnowsze dostępne dane dla jednego wskaźnika z pliku CSV.
    Pomija wiersze z break-in-time-series (OBS_FLAG='b') i brakujące wartości.
    
    Args:
        path (str): ścieżka do pliku CSV
        indicator_name (str): nazwa wskaźnika w kolumnie 'Urban audit indicator'
        value_col_name (str): nowa nazwa kolumny z wartościami
    
    Returns:
        pd.DataFrame: DataFrame z kolumnami ['Geopolitical entity (declaring)', value_col_name]
    """
    df = pd.read_csv(path, delimiter=',')
    df = df[df['Urban audit indicator'] == indicator_name].copy()

    # Pomijamy break-in-time-series i brakujące wartości
    if 'OBS_FLAG' in df.columns:
        df = df[df['OBS_FLAG'] != 'b']
    df = df.dropna(subset=['OBS_VALUE'])

    # Sortujemy malejąco po roku i wybieramy najnowszy wiersz dla każdego miasta
    df = df.sort_values(['Geopolitical entity (declaring)', 'TIME_PERIOD'], ascending=[True, False])
    df = df.drop_duplicates(subset=['Geopolitical entity (declaring)'], keep='first')

    # Zmieniamy nazwę kolumny z wartościami
    return df[['Geopolitical entity (declaring)', 'OBS_VALUE']].rename(columns={'OBS_VALUE': value_col_name})

In [3]:
def load_latest_indicators_wide(
    path,
    indicators,
    indicator_col='Urban audit indicator',
    entity_col='Geopolitical entity (declaring)',
    time_col='TIME_PERIOD',
    value_col='OBS_VALUE',
    obs_flag_col='OBS_FLAG',
    drop_obs_flag_value='b',
    low_memory=False,
    rename_map=None
):
    """
    Wczytuje najnowsze dostępne dane dla wielu wskaźników i zwraca je w szerokim formacie.

    Args:
        path (str): ścieżka do pliku CSV
        indicators (list[str]): lista nazw wskaźników do pobrania
        indicator_col (str): kolumna z nazwą wskaźnika
        entity_col (str): kolumna z nazwą miasta / jednostki
        time_col (str): kolumna z rokiem
        value_col (str): kolumna z wartościami
        obs_flag_col (str): kolumna z flagami obserwacji
        drop_obs_flag_value (str): wartość oznaczająca break-in-time-series
        rename_map (dict): opcjonalna mapa {stara_nazwa: nowa_nazwa}

    Returns:
        pd.DataFrame: jeden wiersz na miasto, kolumny = wskaźniki
    """
    df = pd.read_csv(path, delimiter=',')

    # wybór wskaźników
    df = df[df[indicator_col].isin(indicators)]

    # czyszczenie danych
    if obs_flag_col in df.columns:
        df = df[df[obs_flag_col] != drop_obs_flag_value]
    df = df.dropna(subset=[value_col])

    # najnowsze dane dla miasto × wskaźnik
    df = df.sort_values(
        [entity_col, indicator_col, time_col],
        ascending=[True, True, False]
    )
    df = df.drop_duplicates(
        subset=[entity_col, indicator_col],
        keep='first'
    )

    # pivot do wide
    df = df.pivot(
        index=entity_col,
        columns=indicator_col,
        values=value_col
    ).reset_index()

    # opcjonalne czytelne nazwy kolumn
    if rename_map is not None:
        df = df.rename(columns=rename_map)

    return df


In [4]:
def add_feature(df, feature_df):
    """
    Dołącza dodatkowe dane (feature) do głównego DataFrame po nazwach miast.
    
    Args:
        df (pd.DataFrame): główny DataFrame
        feature_df (pd.DataFrame): DataFrame z dodatkowymi kolumnami do połączenia
    
    Returns:
        pd.DataFrame: zaktualizowany główny DataFrame
    """
    return df.merge(
        feature_df,
        on='Geopolitical entity (declaring)',
        how='left'
    )

## Wczytanie nazw miast i całkowitej populacji

In [5]:
# Wczytanie wszystkich danych o populacji
pop_total = pd.read_csv('../data/raw/01_population.csv', delimiter=',')

# Wybór wierszy z całkowitą populacją i pominięcie break-in-time-series
pop_total = pop_total[
    (pop_total['Urban audit indicator'] == 'Population on the 1st of January, total') &
    (pop_total['OBS_FLAG'] != 'b')
]

# Sortowanie malejąco po roku i wybór najnowszych danych dla każdego miasta
pop_total = pop_total.sort_values(['Geopolitical entity (declaring)', 'TIME_PERIOD'], ascending=[True, False])
pop_latest = pop_total.drop_duplicates(subset=['Geopolitical entity (declaring)'], keep='first')

# Tworzymy główny DataFrame
df = pop_latest[['Geopolitical entity (declaring)', 'OBS_VALUE']].rename(columns={'OBS_VALUE': 'Population'})

## Dodanie informacji o strukturze wiekowej

In [6]:
age_indicators = [
    'Age dependency ratio (population aged 0-19 and 65 and more to population aged 20-64)',
    'Young-age dependency ratio (population aged 0-19 to population 20-64 years)',
    'Old age dependency ratio (population 65 and over to population 20 to 64 years)',
    'Median population age'
]

age_rename = {
    age_indicators[0]: 'Age_dependency_ratio',
    age_indicators[1]: 'Young_dependency_ratio',
    age_indicators[2]: 'Old_dependency_ratio',
    age_indicators[3]: 'Median_age'
}

age_structure = load_latest_indicators_wide(
    path='../data/raw/02_population_structure.csv',
    indicators=age_indicators,
    rename_map=age_rename
)

df = add_feature(df, age_structure)

## Wczytanie informacji o odsetku obcokrajowców

In [7]:
foreigners = load_latest_indicator(
    path='../data/raw/03_foreigners.csv',
    indicator_name='Foreigners as a proportion of population',
    value_col_name='Share_foreigners'
)

df = add_feature(df, foreigners)

In [8]:
df

,Geopolitical entity (declaring),Population,Age_dependency_ratio,Median_age,Old_dependency_ratio,Young_dependency_ratio,Share_foreigners
0,'s-Gravenhage (greater city),813669,61.1,37.0,26.0,35.2,19.4
1,'s-Hertogenbosch,160757,63.8,40.0,29.8,34.0,6.5
2,A Coruña (greater city),247350,67.9,48.3,40.9,27.0,7.6
3,Aachen,252136,54.1,37.0,28.1,25.5,22.3
4,Aberdeen,228180,53.0,37.1,23.7,29.2,16.5
...,...,...,...,...,...,...,...
827,České Budějovice,93426,68.4,43.4,35.5,32.9,3.0
828,Łomża,60468,55.5,40.8,24.7,30.9,0.2
829,Łódź,664860,65.2,44.9,38.1,27.1,0.2
830,Šiauliai,110331,62.8,43.0,31.8,31.0,11.5


## Wczytanie informacji o śmierterlności noworodków i śmiertelności przed 65 rokiem życia

In [9]:
mortality_indicators = [
    'Infant mortality rate (per 1000 live births)',
    'Number of deaths per year under 65 due to diseases of the circulatory or respiratory systems',
    'Crude birth rate (per 1000 inhabitants)',
    'Crude death rate (per 1000 inhabitants)'
]

mortality_rename = {
    'Infant mortality rate (per 1000 live births)': 'Infant_mortality_rate',
    'Number of deaths per year under 65 due to diseases of the circulatory or respiratory systems':
        'Deaths_under_65_circulatory_respiratory',
    'Crude birth rate (per 1000 inhabitants)': 'Crude_birth_rate',
    'Crude death rate (per 1000 inhabitants)': 'Crude_death_rate'
}

mortality = load_latest_indicators_wide(
    path='../data/raw/04_mortality.csv',
    indicators=mortality_indicators,
    rename_map=mortality_rename
)

df = add_feature(df, mortality)

C:\Users\wrons\AppData\Local\Temp\ipykernel_15920\2239181332.py:30: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, delimiter=',')


In [14]:
mortality.sort_values('Crude_death_rate', ascending=False)

Urban audit indicator,Geopolitical entity (declaring),Crude_birth_rate,Crude_death_rate,Infant_mortality_rate,Deaths_under_65_circulatory_respiratory
830,Wałbrzych,6.05,19.61,3.20,145.0
576,Pernik,6.89,18.61,2.07,112.0
203,Dessau-Roßlau,6.13,18.57,0.00,37.0
560,Pabianice,7.60,18.38,4.23,44.0
875,Łódź,8.17,18.18,3.13,542.0
...,...,...,...,...,...
798,Van,NaN,0.89,NaN,NaN
365,Kars,NaN,0.79,NaN,NaN
690,Siirt,NaN,0.38,NaN,NaN
214,Dublin (greater city),NaN,NaN,NaN,435.0


In [11]:
df['Deaths_under_65_circ_resp_per_100k'] = (
    df['Deaths_under_65_circulatory_respiratory'] / df['Population'] * 100_000
)

df = df.drop('Deaths_under_65_circulatory_respiratory', axis=1)

In [13]:
# po analizie irracjonalnych przypadków wychodzi na to, że prawdopodobnie wprowadzono wartości zgonów na 1 mln zamiast na 1 tys. urodzeń
df['Infant_mortality_rate'] = df['Infant_mortality_rate'].apply(lambda x: x/1000 if x >1000 else x)

,count,mean,std,min,25%,50%,75%,max
Population,832.0,249113.379808,599366.813928,37960.00,80938.000000,127602.000000,220709.00000,1.035371e+07
Age_dependency_ratio,825.0,66.331030,8.824602,41.20,61.000000,66.100000,72.00000,1.309000e+02
Median_age,781.0,42.299104,4.320555,17.00,39.900000,42.700000,45.10000,5.420000e+01
Old_dependency_ratio,825.0,32.272606,7.196183,4.50,27.700000,32.000000,36.20000,6.950000e+01
Young_dependency_ratio,825.0,34.104606,6.971661,22.40,29.200000,32.400000,38.40000,1.264000e+02
Share_foreigners,793.0,10.345523,8.182876,0.00,4.100000,9.100000,14.70000,5.820000e+01
Crude_birth_rate,829.0,8.947491,2.727243,0.01,7.200000,8.890000,10.45000,4.229000e+01
Crude_death_rate,828.0,10.743321,2.635752,2.66,8.750000,10.585000,12.34250,1.961000e+01
Infant_mortality_rate,818.0,3.690031,2.292679,0.00,2.180000,3.480000,4.76750,1.528000e+01
Deaths_under_65_circ_resp_per_100k,795.0,43.092267,26.833775,0.00,25.204906,35.775509,53.56663,1.979565e+02
